# Phan Tich Toan Dien LBPH Model


In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display, Image
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
LOGS_DIR = Path('../logs/LBHP')
print(list(LOGS_DIR.glob('*')))


In [ ]:
with open(LOGS_DIR / 'metadata.json', 'r') as f: metadata = json.load(f)
with open(LOGS_DIR / 'confidence_stats.json', 'r') as f: conf_stats = json.load(f)
with open(LOGS_DIR / 'threshold_history.json', 'r') as f: threshold_history = json.load(f)
print(f'Num classes: {metadata["num_classes"]}, Test acc: {metadata["test_accuracy"]*100:.2f}%')


## Model Configuration


In [ ]:
print('='*60)
print('LBPH MODEL CONFIGURATION')
print('='*60)
params = metadata['model_params']
print(f'Radius: {params["radius"]}, Neighbors: {params["neighbors"]}')
print(f'Grid: {params["grid_x"]}x{params["grid_y"]}')
print(f'Num Classes: {metadata["num_classes"]}')
print(f'Threshold: {metadata["threshold"]}')
print(f'Test Accuracy: {metadata["test_accuracy"]*100:.2f}%')
print(f'Test Coverage: {metadata["test_coverage"]*100:.2f}%')


## Threshold Analysis


In [ ]:
th_df = pd.DataFrame(threshold_history)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].plot(th_df['threshold'], th_df['accuracy']*100, 'b-o', lw=2)
axes[0].set_xlabel('Threshold'); axes[0].set_ylabel('Accuracy (%)')
axes[0].set_title('Accuracy vs Threshold'); axes[0].grid(True)
ax2 = axes[1]
ax2.plot(th_df['threshold'], th_df['accuracy']*100, 'b-o', lw=2, label='Accuracy')
ax2_twin = ax2.twinx()
ax2_twin.plot(th_df['threshold'], th_df['coverage']*100, 'g-s', lw=2, label='Coverage')
ax2.set_xlabel('Threshold'); ax2.set_ylabel('Accuracy (%)', color='b')
ax2_twin.set_ylabel('Coverage (%)', color='g')
ax2.set_title('Acc vs Coverage Trade-off'); ax2.grid(True)
plt.tight_layout()
plt.savefig(LOGS_DIR / 'threshold_analysis_comprehensive.png', dpi=150)
plt.show()
display(th_df)


## Confidence Distribution


In [ ]:
print('Correct:', conf_stats['correct'])
print('Incorrect:', conf_stats['incorrect'])


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
cm, cs = conf_stats['correct']['mean'], conf_stats['correct']['std']
im, ist = conf_stats['incorrect']['mean'], conf_stats['incorrect']['std']
x = np.linspace(0, 150, 500)
yc = (1/(cs*np.sqrt(2*np.pi))) * np.exp(-0.5*((x-cm)/cs)**2)
yi = (1/(ist*np.sqrt(2*np.pi))) * np.exp(-0.5*((x-im)/ist)**2)
ax.fill_between(x, yc, alpha=0.5, color='green', label=f'Correct (mean={cm:.1f})')
ax.fill_between(x, yi, alpha=0.5, color='red', label=f'Incorrect (mean={im:.1f})')
ax.axvline(x=metadata['threshold'], color='black', linestyle='--', lw=2, label='Threshold')
ax.set_xlabel('Distance'); ax.set_ylabel('Density')
ax.set_title('LBPH Distance Distribution'); ax.legend(); ax.grid(True)
plt.tight_layout()
plt.savefig(LOGS_DIR / 'confidence_distribution_analysis.png', dpi=150)
plt.show()


## Existing Visualizations


In [ ]:
plots_dir = LOGS_DIR / 'plots'
if plots_dir.exists():
    for img in plots_dir.glob('*.png'): display(Image(filename=str(img), width=700))


In [ ]:
summary = {'model': 'LBPH', 'params': metadata['model_params'],
           'num_classes': metadata['num_classes'], 'threshold': metadata['threshold'],
           'test_accuracy': metadata['test_accuracy']*100, 'test_coverage': metadata['test_coverage']*100}
with open(LOGS_DIR / 'comprehensive_summary.json', 'w') as f: json.dump(summary, f, indent=2)
print(json.dumps(summary, indent=2))
